In [7]:
# --- 0. TensorFlow 호환 모드 설정 및 라이브러리 임포트 ---
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()

import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from alibi.explainers import CEM

# --- 1. 데이터 로드 및 모델 학습 ---
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
feature_names = X.columns.tolist()
target_names = iris.target_names

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 다중 클래스 분류 모델 학습
model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)


# --- 2. 설명할 인스턴스 선택 ---
# 예시: 모델이 'versicolor'(Class 1)로 예측한 첫 번째 테스트 데이터
instance_to_explain = X_test.iloc[[0]].values
original_prediction_idx = model.predict(instance_to_explain)[0]
print(f"설명 대상 샘플의 원본 예측: Class {original_prediction_idx} ({target_names[original_prediction_idx]})")
print("="*50)


# --- 3. Alibi CEM 설명기 생성 ---
# CEM에 필요한 파라미터들을 설정합니다.
# mode: 'PP'(Pertinent Positive) 또는 'PN'(Pertinent Negative)
mode = 'PP'

# shape: 입력 데이터의 형태
shape = (1, X_train.shape[1])

# kappa: 예측 신뢰도. 높을수록 더 확실한 설명을 찾으려 합니다.
kappa = 0.0

# beta: L1 정규화 계수. 변경되는 피처 수를 줄입니다.
beta = .1

# c_init, c_steps: 최적화 상수 및 스텝. 높을수록 더 강하게 탐색합니다.
c_init = 1.0
c_steps = 10

# CEM 설명기 생성
cem_explainer = CEM(
    model.predict_proba,
    mode=mode,
    shape=shape,
    kappa=kappa,
    beta=beta,
    c_init=c_init,
    c_steps=c_steps
)
cem_explainer.fit(X_train.values, no_info_type='median')


# --- 4. 카운터팩추얼 설명 생성 ---
print("CEM 설명을 탐색합니다...")

# explain 메서드에는 목표 클래스를 지정하지 않습니다.
# 생성 시 mode에 따라 PP 또는 PN을 찾습니다.
scaler = StandardScaler()
explanation = cem_explainer.explain(instance_to_explain, verbose=False)


# --- 5. 결과 확인 ---
if explanation.PP is None and explanation.PN is None:
    print("\n설명을 찾지 못했습니다.")
else:
    print(f"\n--- Pertinent Positive (왜 Class {original_prediction_idx}인가?) ---")
    print("설명: 아래 피처들이 현재 값을 가졌기 때문에 'versicolor'로 예측되었습니다.")
    # PP_as_df()는 원본 대비 변경된 부분만 보여줍니다.
    # PP는 원본과 거의 같지만, 예측에 불필요한 노이즈가 제거된 형태입니다.
    print(explanation.PP_as_df(feature_names=feature_names))

    print("\n--- Pertinent Negative (왜 Class 0이 아닌가?) ---")
    print("설명: 아래 피처들이 현재 값을 가졌기 때문에 'setosa'가 아니라고 예측되었습니다.")
    
    # PN을 찾기 위해 mode를 바꾸고 다시 설명
    cem_explainer.mode = 'PN'
    # 'as_class'는 비교 대상 클래스를 지정합니다.
    explanation_pn = cem_explainer.explain(instance_to_explain, as_class=0)
    print(explanation_pn.PN_as_df(feature_names=feature_names))


c:\Users\admin\anaconda3\lib\site-packages\xgboost\training.py:183: UserWarning: [16:24:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


설명 대상 샘플의 원본 예측: Class 1 (versicolor)
CEM 설명을 탐색합니다...


No PP found!



설명을 찾지 못했습니다.
